# Анализ временных рядов - практика

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

## Данные

В этой практике работаем с пятилетней историей посещения портала [statforecasting.com](https://regressit.com/statforecasting.com/). 

Описание данных:
- **day** - день недели
- **day_of_week** - номер дня недели
- **date** - дата в формате год-месяц-день
- **page_loads** - количество загрузок страницы
- **unique_visits** - количество уникальных посетителей
- **first_time_visits** - количество новых посетителей
- **returning_visits** - количество неновых посетителей

Данные прошли небольшую предобработку и немного отличаются от [оригинала](https://www.kaggle.com/datasets/bobnau/daily-website-visitors):
- внесены косметические правки в названиях колонок и в стиль написания чисел
- проверили, что нет пропусков в временном ряду

Мысли по этим данным:
- Данные подвержены сложной сезонности: как внутри недели, так и помесячно.
- Можно обучить модели прогнозировать трафик на 1 день вперед, на 7 дней вперед (на каждый день) или прогноз в целом на неделю.
- Подходы, рассмотренные здесь, применимы для как для прогноза трафика других сервисов, так и в целом для прогнозов любых временных рядов

Прогнозируем количество загрузок страниц

Считаем данные и убедимся, что все в порядке!

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/elentevanyan/time_series_analysis/main/daily_website_visitors_prep.csv')
df.head()

Что может быть не в порядке?

- пропуски: разные значения числа строк и записей в колонках
- типы данных: категории могут быть считаны как числа и наоборот, даты - тексты

In [ ]:
df.info()

Для удобства переведем колонку даты в тип datetime. Тогда, если понадобится, сможем в 1-2 шага делать различные манипуляции с колонкой или создавать из нее новые: находить год, месяц, неделю готовыми командами.

Есть несколько способов смены типа колонки. Чтобы не подгружать дополнительные библиотеки, вытащим из пандас-сундука функцию:

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.dtypes

## Изучение данных

На лекции обсуждали, что временные ряды:
- изучают визуально
- раскладывают на компоненты
- используют для прогноза будущих значений

В датасете не один временной ряд, а несколько - т.н. многомерный ряд.
Для удобства в изучении данных выделим эти колонки в отдельную переменную:

In [ ]:
series_cols = ['page_loads', 'unique_visits', 'first_time_visits', 'returning_visits']

Посмотрим на основные **описательные статистики**:
- количество наблюдений
- среднее
- стандартное отклонение
- минимум
- нижнюю квартиль (25-ый перцентиль)
- медиану (50-ый перцентиль)
- верхнюю квартиль (75-ый перцентиль)
- максимум

In [ ]:
df[series_cols].describe().T

Как будто бы бросающихся в глаза особенностей нет.

Интуитивно кажется, что временные ряды в этом датасете связаны между собой. <br>
Давайте посчитаем корреляции и проверим, есть ли основания этой интуиции.

Считаем матрицу корреляций, где каждая ячейка - корреляция между признаков. Чтобы не скучно было ее интерпретировать, визуализируем сразу в виде тепловой карты:

In [ ]:
mask = np.triu(np.ones_like(df[series_cols].corr(), dtype=bool))
sns.heatmap(df[series_cols].corr(), annot=True, mask=mask)
plt.show()

Видим, что данные сильно похожи между собой! Зная один ряд, мы почти наверняка восстановим и другой ряд. 
Значит, не нужно думать о сложных методах, чтобы прогнозировать многомерные ряды.

А как выглядит исследуемый временной ряд?
Построим подневную кривую и попробуем "на глаз" оценить:

- среднее: если "мысленно" провести линию среднего, какая она - стабильная, растущая, убывающая?
- разброс: как бы вы описали характер разброса данных по этой картинке?
- есть ли какие-то паттерны?

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(data=df, x='date', y='page_loads')
plt.show()

А если посмотреть на недельные данные? Месячные? Может быть, увидим интересное. Здесь и пригодится трансформация колонки даты - сделаем колонки, по которым сможем посчитать агрегации в нужных срезах.

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.isocalendar().week 

Вопросы к обсуждению остаются те же. Но при этом данных сразу за год нет, их надо собрать самим. С точки зрения техники совершенно не важно, как агрегируем и за какой временной период - суммируем, считаем среднее или медиану за период - код один и тот же по структуре.


В следующих блоках делаем следующее:
- собираем статистику (сумма, среднее, медиана и т.п.) за нужный период (год, месяц, неделя)
- визуализируем кривую

### Годичная агрегация

Начнем с анализа по годам. Как изменялось количество загрузок год от года?

In [ ]:
yearly_data = df.groupby('year')['page_loads'].mean().reset_index()
yearly_data

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(data=yearly_data, x='year', y='page_loads')
plt.show()

### Месячная агрегация

Теперь смотрим на ежемесячные данные. Обратите внимание, что данные за несколько лет. Агрегация только по месяцу смешает данные за разные годы, поэтому делаем агрегацию по году и месяцу

In [ ]:
monthly_data = df.groupby(['year', 'month'])['page_loads'].mean().reset_index()
monthly_data['index'] = monthly_data['year'].astype(str) + '_' + monthly_data['month'].astype(str)
monthly_data

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(data=monthly_data, x='index', y='page_loads')
plt.xticks(rotation=90)
plt.show()

### Недельная агрегация

In [ ]:
weekly_data = df.groupby(['year', 'week'])['page_loads'].mean().reset_index()
weekly_data['index'] = weekly_data['year'].astype(str) + '_' + weekly_data['week'].astype(str)
weekly_data

In [ ]:
plt.figure(figsize=(30, 10))
sns.lineplot(data=weekly_data, x='index', y='page_loads')
plt.xticks(rotation=90)
plt.show()

### Поиск сезонности

In [ ]:
sns.lineplot(data=df, x='day', y='page_loads', hue='year')

In [ ]:
sns.lineplot(data=df, x='month', y='page_loads', hue='year')

### Скользящее среднее

In [ ]:
moving_avg = df.drop(['date', 'day'], axis=1).rolling(window = 4).mean()[['page_loads']]
plt.figure(figsize=(30, 10))
sns.lineplot(moving_avg)

In [ ]:
moving_avg = df.drop(['date', 'day'], axis=1).rolling(window = 20).mean()[['page_loads']]
plt.figure(figsize=(30, 10))
sns.lineplot(moving_avg)

In [ ]:
moving_avg = df.drop(['date', 'day'], axis=1).rolling(window = 50).mean()[['page_loads']]
plt.figure(figsize=(30, 10))
sns.lineplot(moving_avg)

## Анализ компонент

### Попробовать самим смоделировать тренд и сезонность

место для простого правила оценки тренда

место для простого правила оценки сезонности

место для расчета ошибок

визуализация

### Использование стандартных средств

In [ ]:
tdi = pd.DatetimeIndex(df.date)
df.set_index(tdi, inplace=True)

Рассказ о том, как работает эта часть

In [ ]:
decompose = seasonal_decompose(x=df['page_loads'])
decompose.plot()

plt.show()

Покопаться со всеми в сезонности - как в таблице, так и отдельно порисовать,

## Моделирование

### Разбиение на train - test

In [ ]:
df.index

In [ ]:
division_date = '2020-08-01'

In [ ]:
train_data = df[df.index < division_date]['page_loads']
test_data = df[df.index >= division_date]['page_loads']

In [ ]:
plt.figure(figsize=(30, 10))
sns.lineplot(train_data, color = "blue")
sns.lineplot(test_data, color = "red")
plt.show()

In [ ]:
len(test_data)

### Наивный прогноз_1

In [ ]:
mean_value = train_data.mean()

In [ ]:
naive_preds = [mean_value]*len(test_data)

In [ ]:
naive_preds = pd.Series(naive_preds, index=test_data.index)

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(test_data, color = "red")
sns.lineplot(naive_preds, color = "green")

plt.show()

In [ ]:
print('MAE прогноза:', mae(test_data, naive_preds))
print('R2 модели:', r2_score(test_data, naive_preds))

### Наивный прогноз_2

$y_{t+1} = y_{t}$

In [ ]:
naive_preds_2 = [train_data[-1]]
for i in range(1, len(test_data)):
    naive_preds_2.append(naive_preds_2[i-1])

In [ ]:
naive_preds_2 = pd.Series(naive_preds_2, index=test_data.index)

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(test_data, color = "red")
sns.lineplot(naive_preds_2, color = "green")

plt.show()

In [ ]:
print('MAE прогноза:', mae(test_data, naive_preds_2))
print('R2 модели:', r2_score(test_data, naive_preds_2))

Мораль: простое не поможет!

## Экспоненциальное сглаживание

В модели экспоненциального сглаживания (exponential smoothing) или экспоненциального скользящего среднего берем все предыдущие значения и задаем каждому из наблюдений определенный вес и (экспоненциально) уменьшаем этот вес по мере углубления в прошлое.

$$\hat{y}_{t+1} = \alpha \cdot y_t + (1-\alpha) \cdot \hat{y}_{t}$$

In [ ]:
train_data

In [ ]:
alpha = 0.6
 
# первое значение совпадает со значением временного ряда
exp_smoothing = [train_data[0]]
 
# в цикле for последовательно применяем формулу ко всем элементам ряда
for i in range(1, len(train_data)):
                 exp_smoothing.append(alpha *train_data[i] + (1 - alpha) * exp_smoothing[i - 1])


In [ ]:
exp_smoothing = pd.Series(exp_smoothing)
exp_smoothing.index = train_data.index

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(train_data);
sns.lineplot(exp_smoothing)

In [ ]:
exp_smoothing[-2:]

Акцент: прогнозирует  на шаг вперед

А что, если нужно на больше?

### Prophet

In [ ]:
from prophet import Prophet

In [ ]:
train_data.head()

In [ ]:
train_data_prophet = train_data.reset_index().rename(columns={'date': 'ds', 'page_loads': 'y'})

In [ ]:
model_prophet = Prophet()

In [ ]:
model_prophet.fit(train_data_prophet)

In [ ]:
prophet_future = model_prophet.make_future_dataframe(periods=len(test_data))
prophet_future.tail()

In [ ]:
prophet_preds = model_prophet.predict(prophet_future)

Описать, что выдает

In [ ]:
#[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
prophet_preds.head()

Считаем метрики

In [ ]:
prophet_preds_value = prophet_preds[prophet_preds['ds'] >=division_date]['yhat']

In [ ]:
print('MAE прогноза:', mae(test_data, prophet_preds_value))
print('R2 модели:', r2_score(test_data, prophet_preds_value))

Рисуем смертные графики!

In [ ]:
prophet_preds_value.index = test_data.index

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(test_data, color = "red")
sns.lineplot(pd.Series(prophet_preds_value), color = "green")

plt.show()

смотрим на креатив самой либы

In [ ]:
fig1 = model_prophet.plot(prophet_preds)


In [ ]:
fig1 = model_prophet.plot_components(prophet_preds)


In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

In [ ]:
plot_plotly(model_prophet, prophet_preds)


# Бонус: Сложные валидации

# Бонус: Сравнение с ML-моделями

# Бонус: Проверка на стационарность

# Бонус: SARIMA